In [16]:
import os
import glob
os.chdir('/home/andrew/ImageSegmentation/Training')

In [83]:
LIMIT_TO_APPROVED = False
EXPERIMENT = 'SegmentationOfCells'
TRIAL = 5
STORE_DIRECTORY = 'PNG_1024'

BUCKET = 'duennerstore'
OUTPUT = 'segmented_data'

All masks have the name SemImage###_XYZ*.png. We find the masks by recursively searchinf for files of this type. Limit search to folders starting with 'Trial'

In [70]:
file_list_full =[file for file in glob.glob('**/SemImage*_*.png', recursive=True)
        if os.path.dirname(file).startswith('Trial')]

if LIMIT_TO_APPROVED == True:
    approved_file_list = [file.split('/')[-1] for file in glob.glob('Approved/*.png')]
    file_list = [file for file in file_list_full
                 if file.split('/')[-1] in approved_file_list]
else:
    file_list = file_list_full

In [71]:
mask_names  = []
mask_paths  = []
mask_image_ids   = []
mask_ids    = []
trials = []

for file in file_list:
    file_name = file.split('/')[-1]
    mask_names.append(file_name)
    mask_id = file_name.split('_')[-1].strip('.png')
    mask_ids.append(mask_id)
    path = os.path.abspath(file)
    trial = int(path.split('/')[5].strip('Trial'))
    trials.append(trial)
    mask_paths.append(path)
    image_number = int(file_name.split('_')[0].strip('SemImage'))
    mask_image_ids.append(image_number)

    
import pandas as pd
data = {
    'Image_ID': mask_image_ids,
    'Mask_ID': mask_ids,
    'Mask': mask_names,
    'Mask_Path': mask_paths,
    'Trials': trials
}

maskdf = pd.DataFrame(data)


In [72]:
import numpy as np
cdir = os.getcwd()
os.chdir('/home/andrew/ImageSegmentation/Training/Raw_Images_1024')
image_names = glob.glob('SemImage*.png')
os.chdir(cdir)
image_ids = []

for idx,image in enumerate(image_names):
    image_id = int(image.strip('SemImage').strip('.png'))
    image_ids.append(image_id)

image_ids = np.array(image_ids,'int')
image_mask_count = np.zeros((image_ids.shape),'int')
mask_image_paths=[]
mask_array = maskdf.values

masked_images = np.array(image_ids,'int')

mask_names = []
mask_trials = []

for idx,img in enumerate(image_ids):
    mask_images = []
    mask_count=0
    mask_image_rows = np.where(img==mask_array[:,0])
    
    these_mask_names = []
    these_mask_paths = []
    these_mask_trials = []
    
    mask_image_paths.append([])
    mask_names.append([])
    mask_trials.append([])
    
    if len(mask_image_rows[0])>0:
        image_mask_count[idx] = len(mask_image_rows[0])
        for row in mask_image_rows[0]:
            these_mask_names.append(mask_array[row,2])
            these_mask_paths.append(mask_array[row,3])
            these_mask_trials.append(mask_array[row,4])
    else:
        these_mask_names.append('none')
        these_mask_paths.append('N/A')
        these_mask_paths.append('')
    mask_names[idx].append(these_mask_names)
    mask_image_paths[idx].append(these_mask_paths)
    mask_trials[idx].append(these_mask_trials)

image_data = {
    'Image_ID': image_ids,
    'Image_Name': image_names,
    'Num_Masks': image_mask_count,
    'Mask_Names': mask_names,
    'Mask Paths': mask_image_paths,
    'Mask Trials': mask_trials
}

image_DF = pd.DataFrame(image_data,index=image_ids)

# Images with at least one mask

In [73]:
image_DF.loc[image_DF['Num_Masks'] > 0]

,Image_ID,Image_Name,Num_Masks,Mask_Names,Mask Paths,Mask Trials
126,126,SemImage126.png,2,"[[SemImage126_MF328.png, SemImage126_I96MJ.png]]",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[4, 4]]"
175,175,SemImage175.png,3,"[[SemImage175_033.png, SemImage175_031.png, Se...",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[2, 2, 2]]"
156,156,SemImage156.png,2,"[[SemImage156_8728Z.png, SemImage156_F6ZV8.png]]",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[4, 4]]"
25,25,SemImage025.png,3,"[[SemImage025_551.png, SemImage025_552.png, Se...",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[2, 2, 2]]"
6,6,SemImage006.png,2,"[[SemImage006_QIXZD.png, SemImage006_A0JUY.png]]",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[4, 4]]"
50,50,SemImage050.png,4,"[[SemImage050_1080_1252.png, SemImage050_522.p...",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[1, 1, 1, 1]]"
75,75,SemImage075.png,3,"[[SemImage075_083.png, SemImage075_081.png, Se...",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[2, 2, 2]]"
250,250,SemImage250.png,4,"[[SemImage250_1080_1252.png, SemImage250_512.p...",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[1, 1, 1, 1]]"
216,216,SemImage216.png,2,"[[SemImage216_3LERP.png, SemImage216_BKV2J.png]]",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[4, 4]]"
150,150,SemImage150.png,4,"[[SemImage150_1080_1252.png, SemImage150_021.p...",[[/home/andrew/ImageSegmentation/Training/Tria...,"[[1, 1, 1, 1]]"


# Images without Mask

In [74]:
img_no_mask = image_DF.loc[image_DF['Num_Masks'] == 0]
img_no_mask

,Image_ID,Image_Name,Num_Masks,Mask_Names,Mask Paths,Mask Trials
34,34,SemImage034.png,0,[[none]],"[[N/A, ]]",[[]]
235,235,SemImage235.png,0,[[none]],"[[N/A, ]]",[[]]
24,24,SemImage024.png,0,[[none]],"[[N/A, ]]",[[]]
81,81,SemImage081.png,0,[[none]],"[[N/A, ]]",[[]]
101,101,SemImage101.png,0,[[none]],"[[N/A, ]]",[[]]
244,244,SemImage244.png,0,[[none]],"[[N/A, ]]",[[]]
233,233,SemImage233.png,0,[[none]],"[[N/A, ]]",[[]]
85,85,SemImage085.png,0,[[none]],"[[N/A, ]]",[[]]
28,28,SemImage028.png,0,[[none]],"[[N/A, ]]",[[]]
144,144,SemImage144.png,0,[[none]],"[[N/A, ]]",[[]]


In [79]:
images_without_mask = np.sort(img_no_mask.values[:,0])

# Select 50 of the unsorted images

In [80]:
num_images_to_classify = len(images_without_mask)
batch_size = 50
image_selection_gap = num_images_to_classify//batch_size

next_image_list=[]
for img_id in range(50):
    next_image_list.append(images_without_mask[image_selection_gap*img_id])

In [81]:
next_image_list

[4,
 10,
 14,
 18,
 22,
 27,
 31,
 35,
 40,
 44,
 48,
 53,
 57,
 61,
 65,
 70,
 74,
 79,
 83,
 87,
 91,
 95,
 101,
 105,
 109,
 113,
 117,
 121,
 127,
 131,
 135,
 139,
 143,
 147,
 152,
 157,
 161,
 165,
 169,
 173,
 178,
 182,
 187,
 191,
 195,
 199,
 204,
 208,
 212,
 217]

# Generate manifest for next batch

In [84]:
import csv

csv_file = EXPERIMENT+str(TRIAL)+'.csv'

if os.path.exists(csv_file):
    os.remove(csv_file)

with open(csv_file, 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',',  quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['image_url'])
    for img_num in next_image_list:
        img_name = 'SemImage{0:03d}.png'.format(img_num)
        bucket_loc = 'https://s3.amazonaws.com/'+BUCKET+'/'+STORE_DIRECTORY+'/'+img_name
        print(bucket_loc)
        csv_writer.writerow([bucket_loc])


https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage004.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage010.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage014.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage018.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage022.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage027.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage031.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage035.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage040.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage044.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage048.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage053.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage057.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage061.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage065.png
https://s3.amazonaws.com/duennerstore/PNG_1024/SemImage

4